In [1]:
import json

In [2]:
import linecache

In [3]:
#跨句实体位置信息
def generate_position_m(pos1,pos2,pos3, words):
    p1 = []
    p2 = []
    p3 = []
    ps1 = []
    ps2 = []
    ps3 = []
    p11 = []
    p22 = []
    p33 = []
    l1 = pos1
    l2 = pos2
    l3 = pos3

    for i, w in enumerate(words):
        a = i-l1
        b = i-l2
        c = i-l3
        if a > 100:
            a = 100
        if b > 100:
            b = 100
        if c > 100:
            c =100
        if a < -100:
            a = -100
        if b < -100:
            b = -100
        if c < -100:
            c = -100
        
        p11.append(a+101)
        p22.append(b+101)
        p33.append(c+101)
    a = len(words)-len(p11)
    
    if a > 0:
        front = int(a/2)
        back = a-front
        front_vec = [0 for i in range(front)]
        back_vec = [0 for i in range(back)]
        p11 = front_vec + p11 + back_vec
        p22 = front_vec + p22 + back_vec
        p33 = front_vec + p33 + back_vec
    else:
        p11 = p11[:len(words)]
        p22 = p22[:len(words)]
        p33 = p33[:len(words)]
    p1.append(p11)
    p2.append(p22)
    p3.append(p33)
    return p1, p2, p3

In [4]:
#单句实体位置信息
def generate_position_s(pos1,pos2,pos3, words):
    p1 = []
    p2 = []
    p3 = []
    ps1 = []
    ps2 = []
    ps3 = []
    p11 = []
    p22 = []
    p33 = []
    l1 = pos1
    l2 = pos2
    l3 = pos3

    for i, w in enumerate(words):
        a = i-l1
        b = i-l2
        c = i-l3
        if a > 50:
            a = 50
        if b > 50:
            b = 50
        if c > 50:
            c =50
        if a < -50:
            a = -50
        if b < -50:
            b = -50
        if c < -50:
            c = -50
        
        p11.append(a+51)
        p22.append(b+51)
        p33.append(c+51)
    a = len(words)-len(p11)
    
    if a > 0:
        front = int(a/2)
        back = a-front
        front_vec = [0 for i in range(front)]
        back_vec = [0 for i in range(back)]
        p11 = front_vec + p11 + back_vec
        p22 = front_vec + p22 + back_vec
        p33 = front_vec + p33 + back_vec
    else:
        p11 = p11[:len(words)]
        p22 = p22[:len(words)]
        p33 = p33[:len(words)]
    p1.append(p11)
    p2.append(p22)
    p3.append(p33)
    return p1, p2, p3

In [35]:
#从原始数据集中构建数据

In [5]:
import codecs
import re

In [6]:
single_relation_set = {'resistance or non-response':1, 'sensitivity':1, 'response':1, 'resistance':1, 'None':0, }

mul_relation_set = {'None':0, 'sensitivity':1, 'response':2, 'resistance':3, 'resistance or non-response':4, }

In [7]:
rel2id={'None':0,'resistance or non-response':1,'sensitivity':2,'response':3,'resistance':4}

In [8]:
def gen_train_data(data_path):
    i = 0
    single_text =[]#单句数据集
    mul_text = []#多句数据集
    mul_train = []
    sin_train = []
    line_list = []#从句子中获得实体索引位置
    with  open(data_path+'/sentences_2nd') as inf:
        for line in inf:
            line_list.append(line.split('	')[1:4])
    with  open(data_path+'/data_graph') as inf:
        for line in inf:
            content = json.loads(line)
            for item in content:
                print(i)
                sentences = item['sentences']
                entities = item['entities']
                #relation = item['relationLabel']
                origin_text = []
                pos_tag = []
                text_index = []
                dep_arcs = []
                dep_graph = []
                if len(sentences)>1:
                    relation = item['relationLabel']#获得relation
                    for sentence in sentences:#句子中有单句和多句
                        for node in sentence['nodes']:
                            origin_text.append(re.sub('[ \t]', '', node['label'].strip()))#原始文本
                            pos_tag.append(re.sub('[ \t]', '', node['postag'].strip()))#POS标签
                            text_index.append(node['index'])#字符index
                            arc_list = node['arcs']#解析关系
                            node_idx = node['index']#节点索引
                            for arc in arc_list:
                                #if arc['label'].startswith('deparc'):#判断依存关系
                                if arc['toIndex'] != node['index'] and arc['toIndex'] != -1 and (node_idx,arc['toIndex'],0,1) not in dep_graph:
                                    dep_graph.append((node_idx,arc['toIndex'],0,1))#所有语义关系都放到de_graph,de_graph不存在重复关系
                    #获取实体在句子中的位置
                    drug_position = int(line_list[i][0].split(' ')[0])
                    gene_position = int(line_list[i][1].split(' ')[0])
                    variant_position = int(line_list[i][2].split(' ')[0])
                    #print(type(drug_position))
                    drug_pos,gene_pos,var_pos = generate_position_m(drug_position,gene_position,variant_position, origin_text)
                    #获得每个单词相对于实体位置
                    mul_dic={
                        "X":        origin_text,
                        "Y":        [mul_relation_set[relation.strip()]],
                        "Pos1":     drug_pos[0],
                        "Pos2":     gene_pos[0],
                        "Pos3":     var_pos[0],
                        "POS":      pos_tag,
                        "DepEdges": dep_graph
                            }
                    mul_train.append(mul_dic)
                    mul_text.append(origin_text)  
                else:
                    relation = item['relationLabel']
                    for sentence in sentences:
                        for node in sentence['nodes']:
                            origin_text.append(re.sub('[ \t]', '', node['label'].strip()))
                            pos_tag.append(re.sub('[ \t]', '', node['postag'].strip()))
                            text_index.append(node['index'])
                            arc_list = node['arcs']
                            node_idx = node['index']
                            for arc in arc_list:
                                if arc['toIndex'] != node['index'] and arc['toIndex'] != -1 and (node_idx,arc['toIndex'],0,1) not in dep_graph:
                                    dep_graph.append((node_idx,arc['toIndex'],0,1))#所有语义关系都放到de_graph,de_graph不存在重复关系
                    #获取实体在句子中的位置
                    drug_position = int(line_list[i][0].split(' ')[0])
                    gene_position = int(line_list[i][1].split(' ')[0])
                    variant_position = int(line_list[i][2].split(' ')[0])
                    drug_pos,gene_pos,var_pos = generate_position_s(drug_position,gene_position,variant_position, origin_text)
                    #获得每个单词相对于实体位置
                    sin_dic={
                        "X":        origin_text,
                        "Y":        [mul_relation_set[relation.strip()]],
                        "Pos1":     drug_pos[0],
                        "Pos2":     gene_pos[0],
                        "Pos3":     var_pos[0],
                        "POS":      pos_tag,
                        "DepEdges": dep_graph
                            }
                    sin_train.append(sin_dic)
                    single_text.append(origin_text) 
                i = i + 1
    return mul_train,sin_train
                #print(len(sentences))

In [9]:
path = '/home/dutir923/chuyonghe/zhaodi/Nary_relation_data/drug_gene_var/0'

In [10]:
m0,s0 = gen_train_data(path)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [11]:
list_m_0 =[]
for i in m0:
    list_m_0.append(0)
list_s_0 =[]
for i in s0:
    list_s_0.append(0)

In [12]:
list_m_0

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [13]:
path = '/home/dutir923/chuyonghe/zhaodi/Nary_relation_data/drug_gene_var/1'

In [14]:
m1,s1 = gen_train_data(path)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [15]:
list_m_1 =[]
for i in m1:
    list_m_1.append(1)
list_s_1 =[]
for i in s1:
    list_s_1.append(1)

In [16]:
path = '/home/dutir923/chuyonghe/zhaodi/Nary_relation_data/drug_gene_var/2'

In [17]:
m2,s2 = gen_train_data(path)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [18]:
list_m_2 =[]
for i in m2:
    list_m_2.append(2)
list_s_2 =[]
for i in s2:
    list_s_2.append(2)

In [19]:
path = '/home/dutir923/chuyonghe/zhaodi/Nary_relation_data/drug_gene_var/3'

In [20]:
m3,s3 = gen_train_data(path)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [21]:
list_m_3 =[]
for i in m3:
    list_m_3.append(3)
list_s_3 =[]
for i in s3:
    list_s_3.append(3)

In [22]:
path = '/home/dutir923/chuyonghe/zhaodi/Nary_relation_data/drug_gene_var/4'

In [23]:
m4,s4 = gen_train_data(path)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [24]:
list_m_4 =[]
for i in m4:
    list_m_4.append(4)
list_s_4 =[]
for i in s4:
    list_s_4.append(4)

In [25]:
m = m0+m1+m2+m3+m4

In [26]:
list_m = list_m_0+list_m_1+list_m_2+list_m_3+list_m_4

In [27]:
list_s = list_s_0+list_s_1+list_s_2+list_s_3+list_s_4

In [28]:
s = s0+s1+s2+s3+s4

In [29]:
len(s)

2028

In [30]:
m[0]

{'DepEdges': [(0, 1, 0, 1),
  (1, 0, 0, 1),
  (1, 2, 0, 1),
  (1, 15, 0, 1),
  (1, 3, 0, 1),
  (1, 7, 0, 1),
  (2, 1, 0, 1),
  (2, 3, 0, 1),
  (3, 2, 0, 1),
  (3, 4, 0, 1),
  (3, 1, 0, 1),
  (4, 3, 0, 1),
  (4, 5, 0, 1),
  (5, 4, 0, 1),
  (5, 6, 0, 1),
  (5, 7, 0, 1),
  (6, 5, 0, 1),
  (6, 7, 0, 1),
  (7, 6, 0, 1),
  (7, 8, 0, 1),
  (7, 5, 0, 1),
  (7, 1, 0, 1),
  (7, 9, 0, 1),
  (8, 7, 0, 1),
  (8, 9, 0, 1),
  (9, 8, 0, 1),
  (9, 10, 0, 1),
  (9, 7, 0, 1),
  (10, 9, 0, 1),
  (10, 11, 0, 1),
  (11, 10, 0, 1),
  (11, 12, 0, 1),
  (11, 15, 0, 1),
  (12, 11, 0, 1),
  (12, 13, 0, 1),
  (12, 15, 0, 1),
  (12, 14, 0, 1),
  (13, 12, 0, 1),
  (13, 14, 0, 1),
  (14, 13, 0, 1),
  (14, 15, 0, 1),
  (14, 12, 0, 1),
  (15, 14, 0, 1),
  (15, 16, 0, 1),
  (15, 1, 0, 1),
  (15, 11, 0, 1),
  (15, 12, 0, 1),
  (15, 20, 0, 1),
  (15, 33, 0, 1),
  (15, 50, 0, 1),
  (16, 15, 0, 1),
  (16, 17, 0, 1),
  (17, 16, 0, 1),
  (17, 18, 0, 1),
  (17, 20, 0, 1),
  (17, 19, 0, 1),
  (18, 17, 0, 1),
  (18, 19, 0, 1),


In [31]:
voc2id = {}
POS_dic = {}

In [32]:
word_num = 1
pos_num = 1
for i in s:
    for j in i['X']:
        if j not in voc2id:
            voc2id[j] = word_num
            word_num = word_num + 1
    for k in i['POS']:
        if k not in POS_dic:
            POS_dic[k] = pos_num
            pos_num = pos_num + 1
            

In [33]:
len(voc2id)

5540

In [34]:
len(POS_dic)

40

In [35]:
word_num = 5541
pos_num = 41
for i in m:
    for j in i['X']:
        if j not in voc2id:
            voc2id[j] = word_num
            word_num = word_num + 1
    for k in i['POS']:
        if k not in POS_dic:
            POS_dic[k] = pos_num
            pos_num = pos_num + 1

In [36]:
len(voc2id)

17207

In [37]:
len(POS_dic)

42

In [38]:
#处理多标签

In [39]:
for i in m:
    sentence = []
    for j in i['X']:
        sentence.append(voc2id[j])
    i['X'] = sentence
    pos = []
    for k in i['POS']:
        pos.append(POS_dic[k])
    i['POS'] = pos
   
    

In [40]:
for i in m:
    print(i['Y'])

[1]
[1]
[4]
[4]
[4]
[4]
[4]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[4]
[4]
[4]
[4]
[1]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[1]
[1]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[1]
[1]
[1]
[1]
[1]
[1]
[4]
[4]
[4]
[1]
[1]
[4]
[4]
[4]
[1]
[1]
[1]
[1]
[1]
[4]
[4]
[1]
[1]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[1]
[4]
[4]
[4]
[4]
[4]
[1]
[1]
[1]
[1]
[4]
[4]
[1]
[4]
[4]
[4]
[4]
[4]
[1]
[1]
[1]
[1]
[1]
[1]
[4]
[4]
[4]
[4]
[1]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[1]
[4]
[4]
[4]
[1]
[4]
[4]
[1]
[1]
[1]
[4]
[1]
[1]
[4]
[4]
[4]
[4]
[1]
[1]
[1]
[1]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[1]
[1]
[4]
[4]
[1]
[4]
[1]
[1]
[1]
[4]
[4]
[4]
[4]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[1]
[1]
[1]
[1]
[1]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[1]
[1]
[1]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[1]
[1]
[1]
[1]
[1]
[4]
[4]
[4]
[4]
[1]
[1]
[1]
[1]
[1]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[1]
[1]
[1]
[1]
[1]
[1]
[4]
[4]
[4]
[4]
[1]
[1]
[4]
[1]
[1]
[1]
[1]
[1]


[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[4]
[4]
[4]
[4]
[4]
[4]
[1]
[4]
[4]


In [41]:
for i in s:
    sentence = []
    for j in i['X']:
        sentence.append(voc2id[j])
    i['X'] = sentence
    pos = []
    for k in i['POS']:
        pos.append(POS_dic[k])
    i['POS'] = pos
    

In [42]:
s[0]

{'DepEdges': [(0, 1, 0, 1),
  (0, 2, 0, 1),
  (1, 0, 0, 1),
  (1, 2, 0, 1),
  (2, 1, 0, 1),
  (2, 3, 0, 1),
  (2, 0, 0, 1),
  (3, 2, 0, 1),
  (3, 4, 0, 1),
  (3, 5, 0, 1),
  (4, 3, 0, 1),
  (4, 5, 0, 1),
  (5, 4, 0, 1),
  (5, 6, 0, 1),
  (5, 3, 0, 1),
  (5, 8, 0, 1),
  (6, 5, 0, 1),
  (6, 7, 0, 1),
  (7, 6, 0, 1),
  (7, 8, 0, 1),
  (8, 7, 0, 1),
  (8, 9, 0, 1),
  (8, 5, 0, 1),
  (8, 13, 0, 1),
  (9, 8, 0, 1),
  (9, 10, 0, 1),
  (10, 9, 0, 1),
  (10, 11, 0, 1),
  (10, 13, 0, 1),
  (11, 10, 0, 1),
  (11, 12, 0, 1),
  (12, 11, 0, 1),
  (12, 13, 0, 1),
  (13, 12, 0, 1),
  (13, 14, 0, 1),
  (13, 10, 0, 1),
  (13, 8, 0, 1),
  (13, 16, 0, 1),
  (13, 21, 0, 1),
  (14, 13, 0, 1),
  (14, 15, 0, 1),
  (15, 14, 0, 1),
  (15, 16, 0, 1),
  (16, 15, 0, 1),
  (16, 17, 0, 1),
  (16, 13, 0, 1),
  (16, 19, 0, 1),
  (16, 25, 0, 1),
  (17, 16, 0, 1),
  (17, 18, 0, 1),
  (17, 19, 0, 1),
  (18, 17, 0, 1),
  (18, 19, 0, 1),
  (19, 18, 0, 1),
  (19, 20, 0, 1),
  (19, 17, 0, 1),
  (19, 16, 0, 1),
  (19, 21, 0, 

In [43]:
len(m)

4959

In [44]:
len(s)

2028

In [45]:
senten_len=[]
for i in m:
    senten_len.append(len(i['X']))
    

In [46]:
senten_len=[]
for i in s:
    senten_len.append(len(i['X']))

In [47]:
max(senten_len)

343

In [48]:
import pickle

In [49]:
from sklearn.model_selection import GroupKFold

# X = [0.1, 0.2, 2.2, 2.4, 2.3, 4.55, 5.8, 8.8, 9, 10]
# #y = ["a", "b", "b", "b", "c", "c", "c", "d", "d", "d"]
# groups = [1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
a=[]
gkf = GroupKFold(n_splits=5)
i = 0
for train, test in gkf.split(m, groups=list_m):
    train_l = []
    test_l = []
    for j in train:
        train_l.append(m[j])
    for k in test:
        test_l.append(m[k])
    data={
    "voc2id":  voc2id,
    "rel2id":  {'None':0,'resistance or non-response':1,'sensitivity':2,'response':3,'resistance':4},
    "max_pos": 446,
    'train':train_l,
    'test':test_l,
    }
    pickle.dump(data, open("tmmul{}.pkl".format(i), 'wb'))
    i = i + 1
    a.append(train)
#     print("%s %s" % (train, test))

In [50]:
m[1000]

{'DepEdges': [(0, 1, 0, 1),
  (0, 6, 0, 1),
  (1, 0, 0, 1),
  (1, 2, 0, 1),
  (1, 6, 0, 1),
  (2, 1, 0, 1),
  (2, 3, 0, 1),
  (2, 6, 0, 1),
  (3, 2, 0, 1),
  (3, 4, 0, 1),
  (3, 6, 0, 1),
  (4, 3, 0, 1),
  (4, 5, 0, 1),
  (4, 6, 0, 1),
  (5, 4, 0, 1),
  (5, 6, 0, 1),
  (6, 5, 0, 1),
  (6, 7, 0, 1),
  (6, 0, 0, 1),
  (6, 1, 0, 1),
  (6, 2, 0, 1),
  (6, 3, 0, 1),
  (6, 4, 0, 1),
  (6, 18, 0, 1),
  (6, 11, 0, 1),
  (7, 6, 0, 1),
  (7, 8, 0, 1),
  (7, 11, 0, 1),
  (8, 7, 0, 1),
  (8, 9, 0, 1),
  (8, 11, 0, 1),
  (9, 8, 0, 1),
  (9, 10, 0, 1),
  (9, 11, 0, 1),
  (10, 9, 0, 1),
  (10, 11, 0, 1),
  (11, 10, 0, 1),
  (11, 12, 0, 1),
  (11, 7, 0, 1),
  (11, 8, 0, 1),
  (11, 9, 0, 1),
  (11, 6, 0, 1),
  (12, 11, 0, 1),
  (12, 13, 0, 1),
  (12, 14, 0, 1),
  (12, 16, 0, 1),
  (13, 12, 0, 1),
  (13, 14, 0, 1),
  (14, 13, 0, 1),
  (14, 15, 0, 1),
  (14, 12, 0, 1),
  (15, 14, 0, 1),
  (15, 16, 0, 1),
  (16, 15, 0, 1),
  (16, 17, 0, 1),
  (16, 12, 0, 1),
  (17, 16, 0, 1),
  (17, 18, 0, 1),
  (18, 17, 

In [52]:
from sklearn.model_selection import GroupKFold

# X = [0.1, 0.2, 2.2, 2.4, 2.3, 4.55, 5.8, 8.8, 9, 10]
# #y = ["a", "b", "b", "b", "c", "c", "c", "d", "d", "d"]
# groups = [1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
a=[]
gkf = GroupKFold(n_splits=5)
i = 0
for train, test in gkf.split(s, groups=list_s):
    train_l = []
    test_l = []
    for j in train:
        train_l.append(s[j])
    for k in test:
        test_l.append(s[k])
    data={
    "voc2id":  voc2id,
    "rel2id":  {'None':0,'resistance or non-response':1,'sensitivity':2,'response':3,'resistance':4},
    "max_pos": 446,
    'train':train_l,
    'test':test_l,
    }
    pickle.dump(data, open("tsmul{}.pkl".format(i), 'wb'))
    i = i + 1
    a.append(train)